In [1]:
import sys
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git")
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git/extern/nes-async-rl/")

In [2]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper

import numpy as np
import matplotlib.pyplot as plt
import itertools
import os.path

In [3]:
from demo_a3c_nes import eval_performance as org_eval_performance

In [4]:
from dqn_phi import dqn_phi
import nes 
from a3c_nes import A3CLSTM
import chainer
from chainer import serializers

In [5]:
example_model_path = "/home/mathia/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5"
rom_path = "/home/mathia/mario.nes"
nr_of_attempts = 30

In [6]:
nesApi = NESInterface(rom_path)
nesApi.render()
nesApi.reset_game()

In [7]:
base_paht = "/home/mathia/checkpoints/{}/{}"
model_names = [
    "scaled_fine_main_reward_1" # coars
    ,"nes_a3c_4" # ehrenbrav
    ,"nes_a3c_scaled_finer_main_reward_1", # main
     "nes_a3c_finer_main_reward_1" # small
    ,"scaled_finer_main_reward_with_points_1" #with points
]

model_chepoint_combos = [(model_name,"16000000_finish.h5")for model_name in model_names]
# model_chepoint_combos.append(("nes_a3c_4","9000000.h5"))
# model_chepoint_combos.append(("scaled_fine_main_reward_1","9400000.h5"))
# model_chepoint_combos.append(("nes_a3c_scaled_finer_main_reward_1","9200000.h5"))
#model_chepoint_combos = itertools.product(model_names, chepoint_filenames) 
model_snapshots = {(directory,checkpoint):base_paht.format(directory,checkpoint) 
                   for (directory,checkpoint) 
                   in model_chepoint_combos}
# make sure every file exists
for path in model_snapshots.values():
    if not os.path.isfile(path):
        print(path)

In [8]:
model_snapshots

{('nes_a3c_4',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_4/16000000_finish.h5',
 ('nes_a3c_finer_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_finer_main_reward_1/16000000_finish.h5',
 ('nes_a3c_scaled_finer_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5',
 ('scaled_fine_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5',
 ('scaled_finer_main_reward_with_points_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_finer_main_reward_with_points_1/16000000_finish.h5'}

In [9]:
#n_actions = nes.NES(hsa.machine_constants.mario_rom_location).number_of_actions

def load_model(weights_path):
    n_actions = nes.n_actions
    model = A3CLSTM(n_actions)
    serializers.load_hdf5(weights_path, model)
    return model
example_model = load_model(example_model_path)

In [10]:
def make_eval_functions():
    return {
        "main_reward": rewards.make_scaled_finer_main_reward(),
        "points": rewards.make_delta_points(),
        "time_left": rewards.reward_for_time_left
    }

In [14]:
def eval_performance(model, nesApi, env, deterministic=False):
    model.reset_state()
    ehrenbrav_sum = 0
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        # repeat action 4 times
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        for i in range(4):
            ehrenbrav_sum += env.receive_action(a)
        
            # TODO Sum of Ehrenbrav, Sum of Main Reward, sum of time left, did it beat the level?, how many deaths
            ram = nesApi.getRAM()
            for name, reward_function in other_rewards.items():
                sums[name] += reward_function(ram)
            if env.is_terminal:
                break
    
    sums["ehrenbrav"] = ehrenbrav_sum
    return sums

In [21]:
eval_performance(example_model,nesApi,nes.NES(rom_path,outside_nes_interface=nesApi,frame_skip=1)
                 ,deterministic=False)

{'ehrenbrav': 7815, 'main_reward': 27.1875, 'points': 0.0, 'time_left': 0}

In [10]:
eval_results = {}
for (experiment_name,model_name), weights in model_snapshots.items():
    model = load_model(weights)
    for attempt_nr in range(nr_of_attempts):
        nesApi.reset_game()
        env = nes.NES(rom_path,outside_nes_interface=nesApi,frame_skip=1)
        performance = eval_performance(model,nesApi,env,deterministic=False)
        print((experiment_name,model_name,attempt_nr), performance)
        eval_results[(experiment_name,model_name,attempt_nr)] = performance

('scaled_finer_main_reward_with_points_1', '16000000_finish.h5', 0) {'main_reward': 60.546875, 'points': 20.0, 'ehrenbrav': 9710, 'time_left': 0}


('scaled_finer_main_reward_with_points_1', '16000000_finish.h5', 1) {'main_reward': 56.328125, 'points': 10.0, 'ehrenbrav': 11295, 'time_left': 0}


('scaled_finer_main_reward_with_points_1', '16000000_finish.h5', 2) {'main_reward': 26.5625, 'points': 20.0, 'ehrenbrav': 7880, 'time_left': 0}


('scaled_finer_main_reward_with_points_1', '16000000_finish.h5', 3) {'main_reward': 26.5234375, 'points': 40.0, 'ehrenbrav': 8110, 'time_left': 0}


('scaled_finer_main_reward_with_points_1', '16000000_finish.h5', 4) {'main_reward': 26.5625, 'points': 40.0, 'ehrenbrav': 6250, 'time_left': 0}


...

('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 25) {'main_reward': 32.3046875, 'points': 40.0, 'ehrenbrav': 6785, 'time_left': 0}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 26) {'main_reward': 6.328125, 'points': 40.0, 'ehrenbrav': 3515, 'time_left': 0}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 27) {'main_reward': 9.7265625, 'points': 0.0, 'ehrenbrav': 2780, 'time_left': 0}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 28) {'main_reward': 35.078125, 'points': 210.0, 'ehrenbrav': 11315, 'time_left': 0}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 29) {'main_reward': 10.4296875, 'points': 10.0, 'ehrenbrav': 6155, 'time_left': 0}


In [11]:
result_df = pandas.DataFrame.from_dict(eval_results,orient="index")

In [14]:
result_df.to_csv("eval_results_5.csv")

In [15]:
result_df.to_pickle("eval_results_5.pickle")

In [19]:
result_df.groupby?

# ladida just rewiting time left

In [10]:
def make_time_left():
    counted_once = False
    def time_left(ram):
        nonlocal counted_once
        if counted_once:
            return 0
        if ram[0x001D] == 3:
            counted_once = True
            return rewards.time_left(ram) 
        return 0
    return  time_left

## Forgot the small main reward 

##org eval

In [22]:
del nesApi

In [24]:
org_eval_performance(rom_path,example_model)

KeyboardInterrupt: 

In [14]:
def close_eval_performance(env, model, deterministic=False,
                     record_screen_path=None):
    #env = nes.NES(rom,record_screen_path=record_screen_path)
    model.reset_state()
    test_r = 0
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        test_r += env.receive_action(a)
    return test_r

In [11]:
#,frame_skip=1
close_eval_performance(nes.NES(rom_path,outside_nes_interface=nesApi),example_model)

43775

### Trying it with a callback

In [15]:
#,frame_skip=1
def wrapped_eval_performance(model):
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    def action_callback(frame_skip_i, ram):
        nonlocal sums
        for name, reward_function in other_rewards.items():
            sums[name] += reward_function(ram)
    eherenbrav_sum = close_eval_performance(
        nes.NES(rom_path, outside_nes_interface=nesApi,
                every_action_callback=action_callback)
        , model)
    sums["eherenbrav"] = eherenbrav_sum
    return sums

In [16]:
wrapped_eval_performance(example_model)

{'eherenbrav': 41990,
 'main_reward': 163.47812500000001,
 'points': 2095.0,
 'time_left': 359}

In [27]:
def frameskip_eval_performance(env, model, deterministic=False,
                     record_screen_path=None):
    #env = nes.NES(rom,record_screen_path=record_screen_path)
    model.reset_state()
    ehrenbrav_sum = 0
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        for i in range(4):
            ehrenbrav_sum += env.receive_action(a)
            ram = env.nes.getRAM()
            if env.is_terminal:
                break
    return ehrenbrav_sum

In [28]:
#,frame_skip=1
frameskip_eval_performance(nes.NES(rom_path,outside_nes_interface=nesApi),example_model)

10855

In [25]:
def frameskip_other_eval_performance(env, model, deterministic=False,
                     record_screen_path=None):
    #env = nes.NES(rom,record_screen_path=record_screen_path)
    model.reset_state()
    ehrenbrav_sum = 0
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        for i in range(4):
            env.nes.render()
            ehrenbrav_sum += env.receive_action(a)
            ram = env.nes.getRAM()
            for name, reward_function in other_rewards.items():
                sums[name] += reward_function(ram)
            if env.is_terminal:
                break

    sums["ehrenbrav"] = ehrenbrav_sum
    return sums

In [26]:
#,frame_skip=1
frameskip_eval_performance(nes.NES(rom_path,outside_nes_interface=nesApi,frame_skip=1),example_model)

{'ehrenbrav': 9845, 'main_reward': 61.09375, 'points': 10.0, 'time_left': 0}

## vergleich

In [ ]:
def eval_performance(model, nesApi, env, deterministic=False):
    model.reset_state()
    ehrenbrav_sum = 0
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        # repeat action 4 times
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        for i in range(4):
            ehrenbrav_sum += env.receive_action(a)
        
            # TODO Sum of Ehrenbrav, Sum of Main Reward, sum of time left, did it beat the level?, how many deaths
            ram = nesApi.getRAM()
            for name, reward_function in other_rewards.items():
                sums[name] += reward_function(ram)
            if env.is_terminal:
                break
    
    sums["ehrenbrav"] = ehrenbrav_sum
    return sums